In [23]:
import torch 
import torch.nn as nn
import torchvision 
import torchvision.transforms as transforms
import torch.functional as F

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [4]:
device = ('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cpu


In [6]:
hidden_size = 100
epochs = 2
batch_size = 32
num_classes = 10
epochs =2
learning_rate = 0.001

In [22]:
transform = transforms.Compose([transforms.ToTensor(), 
                               transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)), 
                               transforms.ToTensor()])

train_dataset = torchvision.datasets.CIFAR10(root= './data/', 
                                             train=True, download=True)

test_dataset = torchvision.datasets.CIFAR10(root= './data/', 
                                             train=False, download=True)

train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size = batch_size,
                                               shuffle = True)

test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size = batch_size,
                                               shuffle = False)

classes = ('plane','car','bird','cat','deer',
           'dog','frog','horse','ship','truck',)

def imshow(imgs):
    imgs = imgs/2 +0.5
    npimgs = imgs.numpy()
    plt.imshow(np.transpose(npimgs, (1,2,0)) )
    plt.show()


#dataiter = iter(train_dataloader)
#images, labels = next(dataiter)
#img_grid = torchvision.utils.make_grid(images[0:25], nrow =5)
#imshow(img_grid)

Files already downloaded and verified
Files already downloaded and verified


In [24]:
#Model 
class Convnet(nn.Module):
    def __init__(self):
        super(Convnet, self).__init__()
        self.layer1 = nn.Conv2d(3,32,3)
        self.pool = nn.MaxPool2d(2,2)
        self.layer2 = nn.Conv2d(32, 64,3)
        self.layer3 = nn.Conv2d(64,64,3)
        self.layer4 = nn.Linear(64*4*4, 3)
        self.layer5 = nn.Linear(64, 10)

    def forward(self, x):
        out = F.relu(self.layer1(x))
        x = self.pool(x)
        x = (F.relu(self.layer2(x)))
        x = self.pool(x)
        x = F.relu(self.layer3(x))
        x = torch.flatten(x,1)
        x = F.relu(self.layer4(x))
        x = self.layer5(x)

        return x
    
model = Convnet().to(device)
print(model)





Convnet(
  (layer1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (layer2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
  (layer3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1))
  (layer4): Linear(in_features=1024, out_features=3, bias=True)
  (layer5): Linear(in_features=64, out_features=10, bias=True)
)


In [25]:
# Define model parameters

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)


In [26]:
## Model training

n_total_steps = len(train_dataloader)

for epoch in range(epochs):
    for i, (image, label) in enumerate(train_dataloader):
        images = image.to(device)
        labels = label.to(device)

        optimizer.zero_grad()
        pred_labels = model(images)
        l = criterion(pred_labels, labels)

        l.backward()
        optimizer.step()

        if epoch%1==0:
            print(f"Epoch {epoch+1}/{epochs} , Step {i+1}/{n_total_steps}, Loss : {l.item():.4f}")


TypeError: default_collate: batch must contain tensors, numpy arrays, numbers, dicts or lists; found <class 'PIL.Image.Image'>